In [ ]:
if False:
    import sys
    !{sys.executable} -m  pip install ipywidgets

In [ ]:
import os
import numpy as np
import time

# make sure the notebook is run from the root folder of maximal-empty-spheres
base_path = os.path.join(os.getcwd())

out_folder = os.path.join(base_path,"output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

# Data

Download Armadillo, Koalla and Bunny meshes

In [ ]:
data_folder = os.path.join(base_path,"data")
if not os.path.exists(data_folder):
    os.mkdir(data_folder)
    
from utils.data import download_meshes
mesh_paths = download_meshes(data_folder)

print(f"\nAvailable Meshes: {list(mesh_paths.keys())}")

camera_views = {"armadillo" : (v_ := np.array([0.,0.,-1.])*2.8,-v_),
                "koala"     : (v_ := np.array([0.4,0.,1.])*2.4,-v_)
}

crops = {
            "armadillo" : [100,150,775,775],
            "koala"     : [100,0,775,775]
}

# GT Sampling

In [ ]:
import numpy as np
def sample_positions(n, random_sampling, sdf):
    if random_sampling:
        U = (np.random.rand(n*n*n, 3)-0.5)*2
    else:
        gx, gy, gz = np.meshgrid(np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1))
        U = np.vstack((gx.flatten(), gy.flatten(), gz.flatten())).T
    U_sdfvals = sdf(U)
    return U, U_sdfvals

import gpytoolbox as gpy

def GT_sdf(mesh_path):
    # Set up gt
    V_gt, F_gt = gpy.read_mesh(mesh_path)
    V_gt = gpy.normalize_points(V_gt)

    s = 0.9 # 0.75
    V_gt *= s/0.5

    # Create and abstract SDF function that is the only connection to the shape
    sdf = lambda x: gpy.signed_distance(x, V_gt, F_gt)[0]
    
    return (V_gt, F_gt), sdf

# Contouring

In [ ]:
from matplotlib.colors import to_rgb
method_colors = {"RFTA":"tab:blue", "MES": "tab:orange", "MC" : "tab:green"}
method_colors["MES_py"] = method_colors["MES"]

In [ ]:
from cgal.EmptySpheresReconstruction import MESReconstruction

# use the same screening weight for RFTA and MES
screening_weight = 1.

from lie_cone import LieConeSDFReconstruction
def MESpy(U,D):
    cone = LieConeSDFReconstruction(np.concatenate([U,D[:,None]],axis=1),
                                        filter_type=3,cut_bbx_factor=1.,filter_results=False,
                                       psr_screening_weight=screening_weight)
    return cone.V,cone.F

method_calls = {
    "RFTA" : lambda U,D,N=None : gpy.reach_for_the_arcs(U,D,screening_weight=screening_weight,parallel=True),
    "MES"  : lambda U,D,N=None : MESReconstruction(U,D,screening_weight=screening_weight,cleanup=True),
    "MC"   : lambda U,D,N      : gpy.marching_cubes(D,U,*[N+1 for n in range(3)]),
    "MES_py" : lambda U,D,N=None: MESpy(U,D)
}

In [ ]:
from tqdm.notebook import tqdm

mesh    = "bunny"
Ns      = [10,20,30,40] # 50,60,70]
methods = ["RFTA",
           "MES",
           "MES_py",
           "MC"]

reconstructions = {m:{} for m in methods}
timings         = {m:{} for m in methods}

# ---------------
for N in tqdm(Ns):
    GT, sdf = GT_sdf(mesh_paths[mesh])
    U,D = sample_positions(N,False,sdf)
    
    for m in tqdm(methods,leave=False):
        if m in method_calls.keys():
            try:
                strt = time.time()
                reconstructions[m][N] = method_calls[m](U,D,N)
                stp  = time.time()
                timings[m][N] = stp-strt
                
            except Exception as e:
                print(f"... {m} failed\n{e}")

In [ ]:
import polyscope as ps
ps.init()
ps.set_up_dir("y_up")
ps.set_ground_plane_mode("none")

screenshots = {m:{} for m in methods}

for m in methods:
    for N in Ns:
        ps.remove_all_structures()
        ps.register_surface_mesh(m,*reconstructions[m][N], color=to_rgb(method_colors[m]))
        if mesh in camera_views.keys():
            ps.look_at(*camera_views[mesh])
        screenshots[m][N] = ps.screenshot_to_buffer()   

In [ ]:
import matplotlib.pyplot as plt
nr,nc = len(methods), len(Ns)
fig, axs = plt.subplots(nr,nc,figsize=(4*nc,4*nr))
if nc==1:
    axs = [[a] for a in axs]
if nr==1:
    axs = [axs]
    
fig.suptitle("recs",fontsize=30)

for mi,m in enumerate(methods):
    for ni,(N,im) in enumerate(screenshots[m].items()):
        ax = axs[mi][ni]
        if ni==0:
            ax.text(0,1300,f"{m}",fontsize=30,rotation=90)
        ax.set_title(f"N=${N}$" ,fontsize=20)
        if mesh in crops.keys():
            u,d,l,r = crops[mesh]
            ax.imshow(screenshots[m][N][u:-(d+1),l:-r])
        else:
            ax.imshow(screenshots[m][N])
        ax.axis("off")
plt.show()

# Mesh Distances

In [ ]:
from utils.mesh_distances import ts_distances

mesh_metrics = []
for m in methods:
    for N in Ns:
        d1,d2,d3 = ts_distances(reconstructions[m][N],GT,1)
        mesh_metrics.append([m,N,timings[m][N],d1,d2,d3])
import pandas as pd
mesh_metrics = pd.DataFrame(mesh_metrics,columns=["Method","N","time","Hausdorff","L1", "Chamfer"])

In [ ]:
metrics = ["time", "Hausdorff","L1", "Chamfer"]

fig,axs = plt.subplots(1,len(metrics),figsize=(3*len(metrics),3))

for _,m in enumerate(methods):
    for i,met in enumerate(metrics):
        df_ = mesh_metrics[mesh_metrics["Method"]==m][["N",met]]
        axs[i].plot(df_["N"],df_[met],label=m)
        axs[i].set_title(met)
        
plt.legend()
plt.show()

In [ ]:
import polyscope as ps


ps.remove_all_structures()
ps.register_surface_mesh("GT", *GT)

if mesh in camera_views.keys():
    ps.look_at(*camera_views[mesh])

ps.show()

In [ ]:
if False:
    
    d1s = squared_distance(GT[0],*reconstructions["MES"][50],use_cpp=True)[0]
    d2s = squared_distance(reconstructions["MES"][50][0],*GT,use_cpp=True)[0]
    
    import polyscope as ps

    ps.remove_all_structures()
    ps.register_surface_mesh("GT", *GT)
    ps.register_point_cloud("m1", GT[0][d1s.argmax(),None,:])
    ps.register_surface_mesh("MES50", *reconstructions["MES"][50])
    ps.register_surface_mesh("MES60", *reconstructions["MES"][60],enabled=False)
    ps.register_point_cloud("m2", reconstructions["MES"][50][0][d2s.argmax(),None,:])

    ps.show()